### Train an MMPose Network
- note that I am using the conda env at /n/groups/datta/tim_sainburg/conda_envs/openmmlab

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from pathlib import Path
import sys
from datetime import datetime

### Download checkpoint for a pretrained model (if desired)
Alternatively, use a previous mouse model as a pretrained model

In [3]:
pretrained_model_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models")
pretrained_model_directory.mkdir(parents=True, exist_ok=True)

In [4]:
# find models here: https://github.com/open-mmlab/mmpose/tree/main/configs
pretrain_model = "rtmpose-m_8xb64-210e_ap10k-256x256"

In [5]:
command = f"source activate {Path(sys.executable).parents[1]}; mim download mmpose --config {pretrain_model} --dest {pretrained_model_directory.as_posix()}"
print(command)

source activate /n/groups/datta/tim_sainburg/conda_envs/mmdeploy; mim download mmpose --config rtmpose-m_8xb64-210e_ap10k-256x256 --dest /n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models


In [6]:
!{command}

/bin/bash: activate: No such file or directory
processing rtmpose-m_8xb64-210e_ap10k-256x256...
rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth exists in /n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models
Traceback (most recent call last):
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1434, in invoke


In [7]:
!ls {pretrained_model_directory.as_posix()}

rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth
rtmdet_s_8xb32-300e_coco.py
rtmpose-m_8xb64-210e_ap10k-256x256.py
rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth


In [8]:
pretrained_model_directory

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models')

### Parameters and dataset

In [9]:
model_name = 'rat_6cam'

# Where the COCO format dataset is located (created in the previous notebook)
dataset_directory = Path("/n/groups/datta/jlove/data/rat_seq/6cam/detection_training/08-26-24-rat_6cam_alldata")

# which config to use (this is what we base the config off of). Should be in the mmpose repo. 
config_loc = Path('/n/groups/datta/tim_sainburg/projects/mmpose/configs/animal_2d_keypoint/rtmpose/ap10k/rtmpose-m_8xb64-210e_ap10k-256x256.py')

# which pretrained model to use (point to .pth file). Pretrained model should be the same model architecture. 
#pretrained_model = pretrained_model_directory / "rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth"
pretrained_model = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-01-05-13-46-05_748568/best_PCK_epoch_230.pth'
use_pretrained_model = False

# working directory (where model output is saved)
output_directory = Path("/n/groups/datta/jlove/data/rat_seq/6cam/mm_training")
formatted_datetime = datetime.now().strftime("%y-%m-%d-%H-%M-%S")
working_directory = (output_directory / 'rtmpose' / f"{model_name}_{formatted_datetime}")
working_directory.mkdir(parents=True, exist_ok=True)

In [10]:
assert config_loc.exists()
assert dataset_directory.exists()

# You shouldn't need to change anything below here
(unless you are using a different skeleton model)

### Display compute / environment info (for future reference)

In [11]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [12]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env
import sys
import mmdet
import torch, torchvision
import mmpose
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info

print(f"Environment: {sys.executable}")
for name, val in collect_env().items():
    print(f'{name}: {val}')
# Check Pytorch installation
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)
print('mmpose version:', mmpose.__version__) 

Environment: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3
sys.platform: linux
Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
CUDA available: True
numpy_random_seed: 2147483648
GPU 0: Quadro RTX 6000
CUDA_HOME: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy
NVCC: Cuda compilation tools, release 12.1, V12.1.105
GCC: gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
PyTorch: 2.1.1
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arc

### Register the new dataset

In [13]:
from mmpipeline.paths import PACKAGE_DIR

In [14]:
from mmpose.registry import DATASETS
from mmpose.datasets.datasets.base import BaseCocoStyleDataset

In [15]:
# this file contains info about the dataset (keypoints, skeleton, etc) needed for traiing
dataset_info_loc =  Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/skeletons/sainburg25pt.py")

In [16]:
@DATASETS.register_module()
class CoCo25pt(BaseCocoStyleDataset):
    METAINFO: dict = dict(from_file=dataset_info_loc)

### Create config file

In [17]:
from mmengine import Config

In [18]:
cfg = Config.fromfile(config_loc.as_posix())

In [19]:
# load COCO pre-trained weight
if use_pretrained_model:
    cfg.load_from = pretrained_model.as_posix()

In [20]:
# set the dataset directory
cfg.data_root = dataset_directory.as_posix()

# set the working directory
cfg.work_dir = working_directory.as_posix()
cfg.randomness = dict(seed=0)

In [21]:
# set dataset configs
cfg.dataset_type = 'CoCo25pt'
cfg.data_mode = 'topdown'

# number of keypoints
cfg.model.head.out_channels = 25

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.ann_file = 'annotations/instances_train.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img='train/')


cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.bbox_file = None
cfg.val_dataloader.dataset.ann_file = 'annotations/instances_val.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img='val/')

cfg.test_dataloader.dataset.type = cfg.dataset_type
cfg.test_dataloader.dataset.bbox_file = None
cfg.test_dataloader.dataset.ann_file = 'annotations/instances_val.json'
cfg.test_dataloader.dataset.data_root = cfg.data_root
cfg.test_dataloader.dataset.data_prefix = dict(img='val/')

# set to custom datset
cfg.train_dataloader.dataset.metainfo = dict(from_file=dataset_info_loc.as_posix())
cfg.val_dataloader.dataset.metainfo = dict(from_file=dataset_info_loc.as_posix())
cfg.test_dataloader.dataset.metainfo = dict(from_file=dataset_info_loc.as_posix())

# set evaluator
cfg.val_evaluator = dict(type='PCKAccuracy')
cfg.test_evaluator = cfg.val_evaluator

cfg.default_hooks.checkpoint.save_best = 'PCK'
cfg.default_hooks.checkpoint.max_keep_ckpts = 15
cfg.default_hooks.checkpoint.interval = 50

cfg.max_epochs = 2000
cfg.train_cfg.max_epochs = 2000

In [22]:
print(cfg)

Config (path: /n/groups/datta/tim_sainburg/projects/mmpose/configs/animal_2d_keypoint/rtmpose/ap10k/rtmpose-m_8xb64-210e_ap10k-256x256.py): {'default_scope': 'mmpose', 'default_hooks': {'timer': {'type': 'IterTimerHook'}, 'logger': {'type': 'LoggerHook', 'interval': 50}, 'param_scheduler': {'type': 'ParamSchedulerHook'}, 'checkpoint': {'type': 'CheckpointHook', 'interval': 50, 'save_best': 'PCK', 'rule': 'greater', 'max_keep_ckpts': 15}, 'sampler_seed': {'type': 'DistSamplerSeedHook'}, 'visualization': {'type': 'PoseVisualizationHook', 'enable': False}, 'badcase': {'type': 'BadCaseAnalysisHook', 'enable': False, 'out_dir': 'badcase', 'metric_type': 'loss', 'badcase_thr': 5}}, 'custom_hooks': [{'type': 'EMAHook', 'ema_type': 'ExpMomentumEMA', 'momentum': 0.0002, 'update_buffers': True, 'priority': 49}, {'type': 'mmdet.PipelineSwitchHook', 'switch_epoch': 180, 'switch_pipeline': [{'type': 'LoadImage', 'backend_args': {'backend': 'local'}}, {'type': 'GetBBoxCenterScale'}, {'type': 'Random

In [23]:
# set preprocess configs to model
cfg.model.setdefault('data_preprocessor', cfg.get('preprocess_cfg', {}))

{'type': 'PoseDataPreprocessor',
 'mean': [123.675, 116.28, 103.53],
 'std': [58.395, 57.12, 57.375],
 'bgr_to_rgb': True}

In [24]:
# save configuration file for future reference
cfg.dump(working_directory / 'config.py')

### run network

In [25]:
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

In [26]:
# build the runner from config
runner = Runner.from_cfg(cfg)

08/26 23:20:21 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Quadro RTX 6000
    CUDA_HOME: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
    PyTorch: 2.1.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;a

KeyError: 'BadCaseAnalysisHook is not in the mmpose::hook registry. Please check whether the value of `BadCaseAnalysisHook` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'

In [27]:
# start training
runner.train()

NameError: name 'runner' is not defined

### The config and path for running inference will be in the working directory